### Model Structure
| Layer             |
| ----------------- |
| `Conv2D(16, 3x3)` |
| `Conv2D(16, 3x3)` |
| `MaxPool2D(2x2)`  |
| `Dropout`         |
| `Conv2D(32, 3x3)` |
| `Conv2D(32, 3x3)` |
| `MaxPool2D(4x4)`  |
| `Dropout`         |
| `Flatten`         |
| `Dense(256)`      |
| `Dropout`         |
| `Dense(1)`        |

![BRAAI CNN Model Structure](images/fig-braai.png)

In [7]:
# Layer Class (template)
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        pass

    def backward(self, d_out, learning_rate):
        pass

In [8]:
import numpy as np
from scipy import signal

class Conv2D:
    def __init__(self, input_shape, kernel_size, output_depth):
        self.input_depth, self.input_height, self.input_width = input_shape
        self.output_depth = output_depth
        self.kernel_size = kernel_size

        self.output_height = self.input_height - kernel_size + 1
        self.output_width = self.input_width - kernel_size + 1

        self.kernels_shape = (output_depth, self.input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape) * np.sqrt(2.0 / np.prod(self.kernels_shape[1:]))  
        self.biases = np.random.randn(output_depth)

    def forward(self, input):
        self.input = input 
        self.output = np.zeros((self.output_depth, self.output_height, self.output_width))

        for i in range(self.output_depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], 'valid')
            self.output[i] += self.biases[i]  

        return self.output

    def backward(self, d_out, learning_rate):
        kernels_gradient = np.zeros_like(self.kernels)
        input_gradient = np.zeros_like(self.input)

        for i in range(self.output_depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], d_out[i], 'valid')
                input_gradient[j] += signal.convolve2d(d_out[i], self.kernels[i, j], 'full')

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * d_out.sum(axis=(1, 2))

        return input_gradient


In [9]:
class ReLu(Layer):
    def forward(self, input):
        self.input = input
        return np.maximum(0, input)

    def backward(self, d_out, learning_rate):
        # Relu prime
        return d_out * (self.input > 0)

In [10]:
class Flatten(Layer):
    def forward(self, x):
        self.input_shape = x.shape
        return x.reshape(-1)

    def backward(self, d_out, learning_rate=None):
        return d_out.reshape(self.input_shape)

In [11]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.zeros(output_size)

    def forward(self, x):
        self.input = x
        return np.dot(self.weights, x) + self.biases

    def backward(self, d_out, learning_rate):
        weights_gradient = np.outer(d_out, self.input)
        input_gradient = np.dot(self.weights.T, d_out)

        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * d_out

        return input_gradient